In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy import stats
import japanize_matplotlib
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import lightgbm as lgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# ===============================================================
# ★★★ オプション設定変数 ★★★
# ===============================================================
GROUP_BY_COURSE = False
GROUP_BY_NUM_HORSES = True

# --- 脚質判定アルゴリズムの選択設定 ---
# "past": 過去の通過順位率の平均値を使用
# "pred": 今回の戦法名(style_name)と指数差による展開予想を使用
STYLE_MODE = "pred"

# ===============================================================
# 関数定義
# ===============================================================
def confidence_interval(mean: float, std: float, n: int, confidence: float = 0.95):
    t_value = stats.t.ppf((1 + confidence) / 2, df=n-1)
    margin = t_value * std / np.sqrt(n)
    return mean - margin, mean + margin

def handle_missing_val(df, features, enabled, min_horses):
    if not enabled:
        print("⚠ 欠損処理スキップ")
        return df

    print("欠損処理中...")
    processed_groups = []
    for race_id, group in df.groupby("race_id"):
        actual_min_horses = group["num_horses"].iloc[0]
        non_missing_mask = group[features].notnull().all(axis=1)
        if non_missing_mask.sum() < actual_min_horses: 
            continue
        for col in features:
            if group[col].isnull().any():
                group[col] = group[col].fillna(group[col].mean(skipna=True))
        processed_groups.append(group)
    df_processed = pd.concat(processed_groups, ignore_index=True)
    print(f"処理後: {len(df_processed)}件（元:{len(df)}）")
    return df_processed

def get_last_corner_position(row):
    for i in [4, 3, 2, 1]:
        val = row.get(f"position_{i}")
        if pd.notnull(val):
            return val
    return np.nan

def create_dl_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid') 
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# ===============================================================
# データ読み込み
# ===============================================================
# target_num_horses = [8, 9, 10, 11, 12]
target_num_horses_int = 8
target_num_horses = [target_num_horses_int]
target_course_ids = [
    102, 103, 201, 202, 301, 302, 402,
    501, 502, 601, 602, 700, 702, 802,
    900, 901, 1001, 1002, 1201, 1304, 1400, 1402
]

input_file = "input_全頭_編集後.csv"
df_all = pd.read_csv(input_file, encoding="utf-8")
df_all["race_date"] = pd.to_datetime(df_all["race_date"])
print(f"全データ: {len(df_all)}件")

# 並び替え
df_all = df_all.sort_values(["race_id", "horse_number"])

# ===============================================================
# A. 過去実績ベースの脚質算出 (running_style_type_past)
# ===============================================================
print("🏇 過去実績ベースの脚質を算出中...")
df_all["last_corner_position"] = df_all.apply(get_last_corner_position, axis=1)
df_all["last_corner_position_rate"] = df_all["last_corner_position"] / df_all["num_horses"]

# 時系列順で過去平均を計算
df_all = df_all.sort_values(["horse_id", "race_date"])
df_all["avg_last_corner_pos_rate_past"] = (
    df_all.groupby("horse_id")["last_corner_position_rate"]
    .transform(lambda x: x.expanding().mean().shift())
)
df_all["running_style_type_past"] = np.where(
    df_all["avg_last_corner_pos_rate_past"] < 0.5, "front",
    np.where(df_all["avg_last_corner_pos_rate_past"].notnull(), "back", np.nan)
)

# ===============================================================
# 対象コース×頭数に絞る & 特徴量基本作成
# ===============================================================
df_target = df_all[df_all["course_id"].isin(target_course_ids)].copy()
df_target = df_target[df_target["num_horses"].isin(target_num_horses)].copy()

handle_missing = True
missing_check_features = ["time_index_average_2", "jockey_place_rate_100"]
df_target = handle_missing_val(df_target, features=missing_check_features, enabled=handle_missing, min_horses=target_num_horses_int)

# タイム指数差（展開予想にも使うためここで作成）
race_avg_ti = df_target.groupby("race_id")["time_index_average_2"].transform("mean")
df_target["time_index_diff_from_avg"] = df_target["time_index_average_2"] - race_avg_ti

# ===============================================================
# B. 展開予想ベースの脚質算出 (running_style_type_pred)
# ===============================================================
print("🏇 展開予想ベースの脚質を算出中...")
style_map = {"逃げ": 1, "先行": 2, "差し": 3, "追込": 4}
df_target["style_rank_val"] = df_target["style_name"].map(style_map).fillna(3)

def predict_front_back(group):
    # 1. 戦法順、2. 指数差(高いほど前) でソート
    group = group.sort_values(["style_rank_val", "time_index_diff_from_avg"], ascending=[True, False])
    n = len(group)
    mid = n // 2
    res = pd.Series(index=group.index, dtype=str)
    res.iloc[:mid] = "front"
    res.iloc[mid:] = "back"
    return res

df_target["running_style_type_pred"] = df_target.groupby("race_id", group_keys=False).apply(predict_front_back)

# ===============================================================
# 使用する脚質カラムの切り替え設定
# ===============================================================
# STYLE_MODE に基づいて、以降の計算で使うメインカラムを決定
USE_STYLE_COL = f"running_style_type_{STYLE_MODE}"
print(f"📢 今回の処理では 【{USE_STYLE_COL}】 を基準に勝率・展開を計算します。")

# ===============================================================
# 特徴量作成 (勝率・相対指標)
# ===============================================================

# (1) 騎手勝率平均との差
race_avg_j = df_target.groupby("race_id")["jockey_place_rate_100"].transform("mean")
df_target["jockey_place_rate_diff_from_avg"] = df_target["jockey_place_rate_100"] - race_avg_j

# (2) 馬番別勝率
df_target["is_win"] = (df_target["finish_rank"] == 1).astype(int)
win_rate_by_horse_number = (
    df_target.groupby(["course_id", "horse_number"])["is_win"]
    .mean().reset_index().rename(columns={"is_win": "win_rate_by_course_horse_number"})
)
df_target = df_target.merge(win_rate_by_horse_number, on=["course_id", "horse_number"], how="left")
race_avg_h = df_target.groupby("race_id")["win_rate_by_course_horse_number"].transform("mean")
df_target["win_rate_by_course_horse_number_diff_from_avg"] = df_target["win_rate_by_course_horse_number"] - race_avg_h

# (3) 選択された脚質タイプ別のコース勝率
# 過去全データから脚質傾向(overall)をマージして算出
horse_mean_style_rate = df_all.groupby("horse_id")["last_corner_position_rate"].mean().reset_index()
horse_mean_style_rate["running_style_type_overall"] = np.where(
    horse_mean_style_rate["last_corner_position_rate"] < 0.5, "front", "back"
)

df_course_all = df_all[df_all["course_id"].isin(target_course_ids)].merge(
    horse_mean_style_rate[["horse_id", "running_style_type_overall"]], on="horse_id", how="left"
)

course_style_win_summary = (
    df_course_all.groupby(["course_id", "running_style_type_overall"])
    .agg(total_runs=("horse_id", "count"), total_wins=("finish_rank", lambda x: (x == 1).sum()))
    .reset_index()
)
course_style_win_summary["win_rate_overall"] = course_style_win_summary["total_wins"] / course_style_win_summary["total_runs"]

# 選択した脚質(USE_STYLE_COL)に対してマスタをマージ
df_target = df_target.merge(
    course_style_win_summary[["course_id", "running_style_type_overall", "win_rate_overall"]],
    left_on=["course_id", USE_STYLE_COL],
    right_on=["course_id", "running_style_type_overall"],
    how="left"
).rename(columns={"win_rate_overall": "running_style_win_prob"}).drop(columns=["running_style_type_overall"])

race_avg_rs = df_target.groupby("race_id")["running_style_win_prob"].transform("mean")
df_target["running_style_win_prob_diff_from_avg"] = df_target["running_style_win_prob"] - race_avg_rs

# (4) 展開（front/backの比率）による勝率
# 選択した脚質カラムに基づいてレース内の比率を計算
race_style_counts = df_target.groupby(["race_id", USE_STYLE_COL]).size().unstack(fill_value=0).reset_index()
for col in ["front", "back"]:
    if col not in race_style_counts.columns: race_style_counts[col] = 0

race_style_counts["total"] = race_style_counts["front"] + race_style_counts["back"]
race_style_counts["ratio_front_round"] = (race_style_counts["front"] / race_style_counts["total"]).round(2)
race_style_counts["ratio_back_round"] = (race_style_counts["back"] / race_style_counts["total"]).round(2)

df_target = df_target.merge(
    race_style_counts[["race_id", "ratio_front_round", "ratio_back_round"]], on="race_id", how="left"
)

# 展開別勝率（ターゲットデータ内で計算）
style_win_ratio_df = df_target.groupby([
    "course_id", USE_STYLE_COL, "ratio_front_round", "ratio_back_round"
]).agg(total_horses=("race_id", "count"), total_wins=("is_win", "sum")).reset_index()
style_win_ratio_df["win_rate"] = style_win_ratio_df["total_wins"] / style_win_ratio_df["total_horses"]

df_target = df_target.merge(
    style_win_ratio_df[["course_id", USE_STYLE_COL, "ratio_front_round", "ratio_back_round", "win_rate"]],
    on=["course_id", USE_STYLE_COL, "ratio_front_round", "ratio_back_round"],
    how="left"
).rename(columns={"win_rate": "style_win_prob_by_ratio"})

race_avg_ratio = df_target.groupby("race_id")["style_win_prob_by_ratio"].transform("mean")
df_target["style_win_prob_by_ratio_diff_from_avg"] = df_target["style_win_prob_by_ratio"] - race_avg_ratio

print("✅ 特徴量作成完了")

全データ: 634887件
🏇 過去実績ベースの脚質を算出中...
欠損処理中...
処理後: 24604件（元:39044）
🏇 展開予想ベースの脚質を算出中...


C:\Users\ryo\AppData\Local\Temp\ipykernel_4228\3477869052.py:147: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_target["running_style_type_pred"] = df_target.groupby("race_id", group_keys=False).apply(predict_front_back)


📢 今回の処理では 【running_style_type_pred】 を基準に勝率・展開を計算します。
✅ 特徴量作成完了


In [37]:
# 1. 各レースに何行（何頭）データがあるかカウントする
race_counts = df_target.groupby("race_id").size()

# 2. 「データの行数」が target_num_horses_int (8) と一致し、
#    かつ「公称の頭数(num_horses)」も 8 であるレースIDを特定する
valid_race_ids = race_counts[
    (race_counts == target_num_horses_int) & 
    (race_counts.index.map(df_target.groupby("race_id")["num_horses"].first()) == target_num_horses_int)
].index

# 3. それらのレースIDだけを抜き出す
df_target = df_target[df_target["race_id"].isin(valid_race_ids)].copy()

print(f"✅ 厳選完了：純粋な{target_num_horses_int}頭立てレースのみ、計 {df_target['race_id'].nunique()} レースに絞り込んだぴょ！")

df_target.to_csv('C:\\Users\\ryo\\Downloads\\df_target.csv', index=False, encoding='cp932')

✅ 厳選完了：純粋な8頭立てレースのみ、計 2582 レースに絞り込んだぴょ！


In [39]:

# ===============================================================
# 展開予想 (running_style_type_pred) の答え合わせ
# ===============================================================
print("\n📊 展開予想の正答率（答え合わせ）を算出中...")

def evaluate_style_prediction(df):
    # 1. 最終コーナー順位の取得 (既存の get_last_corner_position を活用)
    # ※ get_last_corner_position は position_4, 3, 2, 1 の順に最初に見つかった非欠損値を返す
    df["actual_last_corner_pos"] = df.apply(get_last_corner_position, axis=1)
    
    # 最終コーナー順位が取得できないデータは評価から除外
    eval_df = df[df["actual_last_corner_pos"].notnull()].copy()
    
    def label_actual_front_back(group):
        # レース内での最終コーナー通過順位に基づいて、実際の結果(actual_style)を判定
        # ここでも予測時と同じく「昇順ソートして上位半分」を front とする
        group = group.sort_values("actual_last_corner_pos", ascending=True)
        n = len(group)
        mid = n // 2
        
        res = pd.Series(index=group.index, dtype=str)
        res.iloc[:mid] = "front"
        res.iloc[mid:] = "back"
        return res

    # 実際の展開結果をラベル付け
    eval_df["actual_style"] = eval_df.groupby("race_id", group_keys=False).apply(label_actual_front_back)
    
    # 2. 予測と実績の比較
    # 予測(running_style_type_pred) と 実績(actual_style) が一致しているか
    eval_df["is_correct"] = (eval_df["running_style_type_pred"] == eval_df["actual_style"])
    
    # 3. 全体正答率
    overall_accuracy = eval_df["is_correct"].mean()
    
    # 4. 脚質(style_name)ごとの正答率
    style_accuracy = eval_df.groupby("style_name")["is_correct"].mean().sort_values(ascending=False)
    
    # 5. コースごとの正答率（上位5・下位5）
    course_accuracy = eval_df.groupby("course_id")["is_correct"].mean()
    
    return overall_accuracy, style_accuracy, course_accuracy, eval_df

# 実行
accuracy, style_acc, course_acc, result_df = evaluate_style_prediction(df_target)

print("-" * 30)
print(f"✅ 全体正答率: {accuracy:.2%}")
print("-" * 30)
print("■ 脚質(style_name)別の予想正答率:")
print(style_acc.map(lambda x: f"{x:.2%}"))
print("-" * 30)
print("■ コース別正答率（ワースト5）: ※改善のヒントになります")
print(course_acc.sort_values().head(5).map(lambda x: f"{x:.2%}"))
print("-" * 30)

# 混同行列（マトリックス）の表示
confusion_matrix = pd.crosstab(result_df['actual_style'], result_df['running_style_type_pred'], normalize='index')
print("■ 予測の傾向（行：実際 / 列：予測）")
print(confusion_matrix.map(lambda x: f"{x:.2%}"))


📊 展開予想の正答率（答え合わせ）を算出中...
------------------------------
✅ 全体正答率: 68.59%
------------------------------
■ 脚質(style_name)別の予想正答率:
style_name
逃げ    80.06%
追込    75.97%
先行    75.72%
差し    63.49%
Name: is_correct, dtype: object
------------------------------
■ コース別正答率（ワースト5）: ※改善のヒントになります
course_id
702     61.54%
1201    62.50%
1304    63.75%
802     64.29%
700     65.53%
Name: is_correct, dtype: object
------------------------------
■ 予測の傾向（行：実際 / 列：予測）
running_style_type_pred    back   front
actual_style                           
back                     68.56%  31.44%
front                    31.38%  68.62%


C:\Users\ryo\AppData\Local\Temp\ipykernel_4228\265104599.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eval_df["actual_style"] = eval_df.groupby("race_id", group_keys=False).apply(label_actual_front_back)


In [54]:
# --- 設定 ---
num_feat_cols = [
    "time_index_diff_from_avg", 
#    "jockey_place_rate_diff_from_avg",
#    "win_rate_by_course_horse_number_diff_from_avg",
#    "running_style_win_prob_diff_from_avg",
#    "style_win_prob_by_ratio_diff_from_avg"
]
cat_feat_cols = ["horse_id", "jockey_id", "horse_number", "running_style_type_pred"]

# データの準備
# 戻り値: 数値テンソル, カテゴリテンソル(IDが4つ並んでる), 正解ラベル
X_num, X_cat, y_tensor = prepare_race_data(df_target, num_feat_cols, cat_feat_cols)

# 各カテゴリのユニーク数（Embedding層のサイズ）を取得
# X_catは (レース, 8頭, 4種類) の形。各種類の最大値+1をサイズにするぴょ
cat_dims = [X_cat[:,:,i].max().item() + 1 for i in range(len(cat_feat_cols))]

class MultiEmbedScenarioModel(nn.Module):
    def __init__(self, num_dim, cat_dims):
        super().__init__()
        # 各Embedding層の定義
        self.horse_emb = nn.Embedding(cat_dims[0], 16)
        self.jockey_emb = nn.Embedding(cat_dims[1], 8)
        self.num_emb   = nn.Embedding(cat_dims[2], 4)
        self.style_emb = nn.Embedding(cat_dims[3], 2)
        
        # 合計次元数: 数値(5) + 埋め込み(16+8+4+2) = 35
        # 割り切れるように調整（例: 35 -> 36にするために、数値データの扱いを調整 or 埋め込みを調整）
        # 今回は total_dim=36 になるように style_embを3次元にするぴょ！
        self.style_emb = nn.Embedding(cat_dims[3], 3)
        self.total_dim = num_dim + 16 + 8 + 4 + 3 # = 36
        
        # 36なら num_heads=4 で割り切れるぴょ！
        self.attention = nn.MultiheadAttention(embed_dim=self.total_dim, num_heads=4, batch_first=True)
        
        self.head_scenario_A = nn.Linear(self.total_dim, 1)
        self.head_scenario_B = nn.Linear(self.total_dim, 1)
        self.gate = nn.Linear(self.total_dim * target_num_horses_int, 2)

    def forward(self, x_num, x_cat):
        # x_catの各列を取り出して埋め込む
        h_e = self.horse_emb(x_cat[:, :, 0])
        j_e = self.jockey_emb(x_cat[:, :, 1])
        n_e = self.num_emb(x_cat[:, :, 2])
        s_e = self.style_emb(x_cat[:, :, 3])
        
        # 全部くっつけるぴょ！ (数値 + 全埋め込み)
        combined = torch.cat([x_num, h_e, j_e, n_e, s_e], dim=-1)
        
        # 1. 展開の相互作用
        attn_out, _ = self.attention(combined, combined, combined)
        
        # 2. シナリオ別スコア
        score_A = self.head_scenario_A(attn_out).squeeze(-1)
        score_B = self.head_scenario_B(attn_out).squeeze(-1)
        
        # 3. ゲート（展開予測）
        gate_out = self.gate(combined.view(combined.size(0), -1))
        scenario_weights = F.softmax(gate_out, dim=-1)
        
        # 4. 合成
        prob_A = F.softmax(score_A, dim=-1)
        prob_B = F.softmax(score_B, dim=-1)
        final_probs = scenario_weights[:, 0:1] * prob_A + scenario_weights[:, 1:2] * prob_B
        
        return final_probs, scenario_weights

# --- モデル初期化 ---
model = MultiEmbedScenarioModel(len(num_feat_cols), cat_dims)

# --- 試運転 ---
probs, scenario_weights = model(X_num[:5], X_cat[:5])
print("\n🔥 予測完了だぴょ！")
print(f"1着確率:\n{probs[0].detach().numpy()}")
print(f"シナリオ確率 [前, 差]: {scenario_weights[0].detach().numpy()}")

データ整形中: 100%|███████████████████████████████████████████████████████████████| 2582/2582 [00:14<00:00, 175.14it/s]


🔥 予測完了だぴょ！
1着確率:
[0.06421342 0.13377984 0.07168897 0.1503606  0.07781889 0.16019955
 0.15074672 0.19119203]
シナリオ確率 [前, 差]: [0.00165056 0.9983494 ]


In [55]:
# ===============================================================
# 1. データの仕分け（Nを指定）
# ===============================================================
N = 500  # ここに学習に使いたいレース数を指定するぴょ！

# テンソルを分割
X_num_train, X_num_test = X_num[:N], X_num[N:]
X_cat_train, X_cat_test = X_cat[:N], X_cat[N:]
y_train, y_test = y_tensor[:N], y_tensor[N:]

# テスト対象の元のデータ
valid_race_ids = df_target[df_target['num_horses'] == target_num_horses_int]['race_id'].unique()
test_race_ids = valid_race_ids[N:]
df_test_raw = df_target[df_target['race_id'].isin(test_race_ids)].sort_values(['race_id', 'horse_number']).copy()

# ===============================================================
# 2. 学習の実行（トレーニング）
# ===============================================================
model.train()
train_loader = DataLoader(TensorDataset(X_num_train, X_cat_train, y_train), batch_size=32, shuffle=True)

for epoch in range(30):
    for b_num, b_cat, b_y in train_loader:
        optimizer.zero_grad()
        outputs, _ = model(b_num, b_cat)
        loss = criterion(outputs, b_y)
        loss.backward()
        optimizer.step()

# ===============================================================
# 3. 予想と馬券購入シミュレーション（人気別閾値を導入！）
# ===============================================================
# 人気（popularity）ごとの期待値閾値を設定するぴょ！
# 数字を書き換えて戦略を練ってほしいぴょ。
thresholds = {
    1.0: 1.20,
    2.0: 1.50,
    3.0: 2.00,
    4.0: 2.00,
}
default_threshold = 2.00

model.eval()
with torch.no_grad():
    pred_probs, _ = model(X_num_test, X_cat_test)
    pred_probs = pred_probs.numpy().flatten()

df_test_raw['pred_prob'] = pred_probs
df_test_raw['expected_value'] = df_test_raw['pred_prob'] * df_test_raw['odds']

# 人気ごとの閾値を適用するぴょ
df_test_raw['threshold'] = df_test_raw['popularity'].map(thresholds).fillna(default_threshold)

# 各馬の期待値が、その人気の閾値を超えているか判定
df_test_raw['bet_amount'] = np.where(df_test_raw['expected_value'] > df_test_raw['threshold'], 100, 0)

# 的中した時の払い戻し
df_test_raw['return_amount'] = np.where((df_test_raw['bet_amount'] > 0) & (df_test_raw['is_win'] == 1), 
                                        df_test_raw['odds'] * 100, 0)

# ===============================================================
# 4. 人気順（popularity）別の集計（的中数を追加！）
# ===============================================================
summary = df_test_raw[df_test_raw['bet_amount'] > 0].groupby('popularity').agg(
    購入レース数=('race_id', 'nunique'),
    購入馬券数=('bet_amount', 'count'),
    的中数=('is_win', 'sum'), # 的中数を追加したぴょ！
    総投資額=('bet_amount', 'sum'),
    総回収額=('return_amount', 'sum')
).reset_index()

summary['的中率'] = (summary['的中数'] / summary['購入馬券数'] * 100).round(1).astype(str) + '%'
summary['回収率'] = (summary['総回収額'] / summary['総投資額'] * 100).round(1).astype(str) + '%'
summary['採用閾値'] = summary['popularity'].map(thresholds).fillna(default_threshold)

print("\n📊 【シミュレーション結果】人気別成績")
# 的中数と採用閾値を表示項目に加えたぴょ
display_cols = ['popularity', '採用閾値', '購入馬券数', '的中数', '的中率', '回収率', '総投資額', '総回収額']
print(summary[display_cols].to_markdown(index=False))

# 全体成績
total_invest = summary['総投資額'].sum()
total_return = summary['総回収額'].sum()
print(f"\n✨ 全体回収率: {total_return / total_invest:.1%}" if total_invest > 0 else "\n購入条件に合う馬がいなかったぴょ...")


📊 【シミュレーション結果】人気別成績
|   popularity |   採用閾値 |   購入馬券数 |   的中数 | 的中率   | 回収率   |   総投資額 |   総回収額 |
|-------------:|-----------:|-------------:|---------:|:---------|:---------|-----------:|-----------:|
|            2 |        1.5 |           19 |        2 | 10.5%    | 98.4%    |       1900 |       1870 |
|            3 |        2   |           99 |        4 | 4.0%     | 62.3%    |       9900 |       6170 |
|            4 |        2   |          452 |        9 | 2.0%     | 37.4%    |      45200 |      16900 |
|            5 |        2   |         1078 |       31 | 2.9%     | 74.7%    |     107800 |      80560 |
|            6 |        2   |         1688 |       47 | 2.8%     | 80.7%    |     168800 |     136250 |
|            7 |        2   |         1983 |       39 | 2.0%     | 83.2%    |     198300 |     165080 |
|            8 |        2   |         2059 |       10 | 0.5%     | 36.8%    |     205900 |      75780 |

✨ 全体回収率: 65.4%
